All necessary libraries

In [1]:
import pymupdf
import llama_cpp
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import json

c:\Users\arnab\miniconda3\envs\ragEnv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path of pdf

In [2]:
path="D:\\personalCode\\RAG-RUNNER\\documents\\sample.pdf"

In [14]:
path_json="D:\\personalCode\\RAG-RUNNER\\metrics\\sample1.json"

PDF text reading function, text splitting function

In [4]:
def read_pdf_text(path):
    doc=pymupdf.open(path)
    full_text=""
    for page in doc:
        full_text+=page.get_text()
    return full_text

def split_into_chunks(text, chunk_size=500, overlap=50):
    chunks=[]
    for i in range(0, len(text), chunk_size-overlap):
        chunks.append(text[i:i+chunk_size])
    return chunks

In [5]:
text=read_pdf_text(path)
chunks=split_into_chunks(text)

Embedding and storage of vectors

In [6]:
embedder=SentenceTransformer('all-MiniLM-L6-v2')
vectors=embedder.encode(chunks)

In [7]:
dimension=vectors[0].shape[0]
index=faiss.IndexFlatL2(dimension)
index.reset()
index.add(np.array(vectors))

id_to_text={i: chunk for i, chunk in enumerate(chunks)}

In [8]:
def search_chunks(query, top_k=3):
    query_vec=embedder.encode([query])
    D, I=index.search(np.array(query_vec), top_k)
    return [id_to_text[i] for i in I[0]]

Model being used

In [1]:
import llama_cpp

In [12]:
model_path_gguf=r"D:\\personalCode\\RAG-runner-deployment\\backend\\models\\Llama-3.2-8B-Instruct-Q5_K_M.gguf"
model=llama_cpp.Llama(model_path=model_path_gguf, chat_format="llama-2", n_ctx=8192, n_gpu_layers=-1)

llama_model_load_from_file_impl: using device CUDA0 (NVIDIA GeForce RTX 4060 Laptop GPU) - 3535 MiB free
llama_model_loader: loaded meta data with 29 key-value pairs and 292 tensors from D:\\personalCode\\RAG-runner-deployment\\backend\\models\\Llama-3.2-8B-Instruct-Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Llama 3.1 8B Instruct
llama_model_loader: - kv   3:                       general.organization str              = Meta Llama
llama_model_loader: - kv   4:                           general.finetune str              = Instruct
llama_model_loader: - kv   5:                           general.basename str      

In [13]:
print(model.context_params.n_ctx) 

8192


In [14]:
query="Hello, I am arnab, a CSE major doing Bachelors in technology."

tamil_prompt=f"""<|im_start|>system
You are a dedicated english to tamil translator, whose job is to translate the user's {query} into tamil language, using tamil characters.
Ensure the output is only in tamil, and is grammatically correct.
Do not add unnecessary decorators.
<|im_end|>
<|im_start|>user

Question:
{query}
<|im_end|>
<|im_start|>assistant
"""

temp=0.7
max_tokens=512

response=model.create_completion(
    prompt=tamil_prompt,
    temperature=temp,
    max_tokens=max_tokens
)

assistant_reply=response['choices'][0]['text']
assistant_reply=assistant_reply.replace("[/INST]", "")
print("\n-----------------------------------\n")
print(assistant_reply)
print("\n-----------------------------------\n")

llama_perf_context_print:        load time =    1586.90 ms
llama_perf_context_print: prompt eval time =    1586.13 ms /   122 tokens (   13.00 ms per token,    76.92 tokens per second)
llama_perf_context_print:        eval time =   10062.59 ms /   129 runs   (   78.00 ms per token,    12.82 tokens per second)
llama_perf_context_print:       total time =   11788.48 ms /   251 tokens



-----------------------------------

வணக்கம், நான் அர்நாப், கணினி அறிவியல் மற்றும் பொறியியல் துறையில் பட்டதாரி படிப்பை செய்கிறேன். <|im_end|>

-----------------------------------



In [9]:
assistant_reply=response['choices'][0]['text']
assistant_reply=assistant_reply.replace("[/INST]", "")
print(assistant_reply)

 (AI) and its impact on various industries.

நான் அறிந்துள்ளது பொருள் நிலையில் இரு அணுக்கைகள் உண்டு. முதல் அணுக்கை என்று அறியப்படும் இயக்கம் இன்றளவு வாழ்கிறது. இது இயக்கத்தின் ஒரு வடிவம் ஆகும். இது வாழ்கிறது என்பதற்கு காரணம் இயக்கத்தின் நிலைவு ஆகும். இயக்கத்தின் நிலைவு இயக்கத்தின் வடிவங்களில் இரு வடிவங்கள் உண்டு. இயக்கம் வாழ்கிறது என்பதற்கு காரணம் இயக்கத்தின் நிலைவு ஆகும். இயக்கம் வாழ்கிறது என்பதற்கு காரணம் இயக்கத்தின் நிலைவு


User input and search

In [11]:
user_query=input("Please enter query: ")
context="\n\n".join(search_chunks(user_query))

context_tokens = model.tokenize(context.encode("utf-8"))
context_token_count = len(context_tokens)
print(f"Total tokens used by the context: {context_token_count}")

Total tokens used by the context: 437


In [15]:
with open(path_json, 'r') as f:
    data = json.load(f)

print(data)

{'student': {'id': 'S12345', 'name': 'Arnab', 'age': 19, 'date': '2025-05-20'}, 'test_context': {'type': 'Fitness Evaluation', 'source_pdf': 'fitness_assessment_protocols_vr2025.pdf', 'description': 'VR simulation mimicking physical fitness tests based on standard protocols. Includes a theoretical quiz.'}, 'vr_simulation_results': {'session_duration_seconds': 900, 'metrics': {'cardio_endurance_score': 78, 'muscular_strength_score': 84, 'reaction_time_ms': 245, 'balance_score': 90, 'flexibility_score': 72, 'fatigue_index': 0.18, 'errors_made': 1, 'tasks_completed': 10, 'avg_heart_rate_bpm': 138}, 'engagement_level': 'high', 'instructor_comments': 'Impressive balance and strength. Could improve flexibility routines.'}, 'quiz_results': {'topic': 'Basic Fitness & Anatomy', 'total_questions': 15, 'correct_answers': 13, 'score_percentage': 86.7, 'time_taken_seconds': 410, 'difficulty_level': 'medium', 'remarks': 'Strong grasp on physiology; missed a couple on stretching techniques.'}, 'rag_a

In [16]:
final_prompt=f"""<|im_start|>system
You are a helpful assistant in a set up, where the task is generate evaluation feedback. Consider the metrics to contain information on the conducted evaluation. Use the added context, to enhance the answers created from the metrics. If the answer is not present in the context, print "Insufficient context" and nothing else. Structure your response in markdown, using bullet points or headings if appropriate. Ensure that if there is no relevant information, you provide "Insufficient context" and nothing else at all. <|im_end|>
<|im_start|>user
Use the following metrics to answer the question. Enhance the answer using the given context.

Metrics:
{data}

Context:
{context}

Question:
{user_query}<|im_end|>
<|im_start|>assistant
"""

temp=0.7
max_tokens=512

response=model.create_completion(
    prompt=final_prompt,
    temperature=temp,
    max_tokens=max_tokens
)

llama_perf_context_print:        load time =     774.31 ms
llama_perf_context_print: prompt eval time =     764.54 ms /   939 tokens (    0.81 ms per token,  1228.18 tokens per second)
llama_perf_context_print:        eval time =    6906.68 ms /   511 runs   (   13.52 ms per token,    73.99 tokens per second)
llama_perf_context_print:       total time =    8325.97 ms /  1450 tokens


In [17]:
assistant_reply=response['choices'][0]['text']
assistant_reply=assistant_reply.replace("[/INST]", "")
print(assistant_reply)

Markdown:

**How to do a Push Up?**

- **Positioning**: 
  - Begin in a high plank position with your hands shoulder-width apart. 
  - Keep your feet together and your body in a straight line, with your head directly in line with your hands.
  - Your body should form a straight line from your head to your feet, with a slight bend in your knees.

- **Engagement**: 
  - **Shoulders**: 
    - Pull your shoulder blades together and back to engage your upper back and shoulder muscles.
    - Imagine you’re trying to touch the opposite shoulder with your elbow.
    - **Pectorals**: 
    - Keep your chest engaged by pulling your chest muscles together and up.
    - **Core**: 
    - Tighten your abdominal muscles to protect your lower back.
    - **Triceps and Deltoids**: 
    - Maintain a straight arm position, bending your elbows just enough to form a 90-degree angle at the top of your movement.

- **Movement**: 
  - **Lower**: 
    - Start by lowering your body until your chest is just above

In [19]:
hallucination_prompt = f"""<|im_start|>system
You are an expert fact-checking assistant. Your job is to verify whether the assistant's answer is fully supported by the given context. If parts of the answer are not present in the context, clearly identify them. Be strict and objective in your judgment.<|im_end|>
<|im_start|>user
Context:
{context}

Answer:
{assistant_reply}

Task:
Determine whether the answer is hallucinated. List any parts of the answer that are not supported by the context. If answer is determined to not be hallucinated respond with "Assistant reply is not hallucinated."<|im_end|>
<|im_start|>assistant
"""

hal_response = model.create_completion(
    prompt=hallucination_prompt,
    temperature=temp,
    max_tokens=256
)
assistant_hallucination = hal_response['choices'][0]['text']
print(assistant_hallucination)

Llama.generate: 6 prefix-match hit, remaining 605 prompt tokens to eval
llama_perf_context_print:        load time =     778.50 ms
llama_perf_context_print: prompt eval time =     336.30 ms /   605 tokens (    0.56 ms per token,  1799.01 tokens per second)
llama_perf_context_print:        eval time =     117.94 ms /     7 runs   (   16.85 ms per token,    59.35 tokens per second)
llama_perf_context_print:       total time =     462.76 ms /   612 tokens


Assistant reply is not hallucinated.


In [15]:
print(assistant_hallucination)

Assistant reply is not hallucinated.


In [20]:
import numpy as np
from sklearn.cluster import AgglomerativeClustering
from sentence_transformers import SentenceTransformer
from collections import Counter
import math

In [17]:
def compute_semantic_entropy(responses, distance_threshold=1.0):
    embeddings=embedder.encode(responses)

    clustering=AgglomerativeClustering(n_clusters=None, distance_threshold=distance_threshold, linkage="average")
    labels=clustering.fit_predict(embeddings)

    counts=Counter(labels)
    total=sum(counts.values())
    probabilities=[count/total for count in counts.values()]
    entropy=-sum(p*math.log2(p) for p in probabilities)

    return entropy, labels    

In [18]:
for i in range(20):
    semantic_response=[model.create_completion(prompt=user_query, temperature=temp, max_tokens=256)['choices'][0]['text'] for _ in range(5)]
    entropy, labels=compute_semantic_entropy(semantic_response, distance_threshold=1.0)
    print(f"At iteration {i} Entropy:{entropy}")
    if entropy>=1.5:
        print(f"iteration {i} detected possible hallucination")

Llama.generate: 1 prefix-match hit, remaining 7 prompt tokens to eval
llama_perf_context_print:        load time =     737.43 ms
llama_perf_context_print: prompt eval time =     222.18 ms /     7 tokens (   31.74 ms per token,    31.51 tokens per second)
llama_perf_context_print:        eval time =    1908.22 ms /   138 runs   (   13.83 ms per token,    72.32 tokens per second)
llama_perf_context_print:       total time =    2259.40 ms /   145 tokens
Llama.generate: 7 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =     737.43 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =    2588.46 ms /   187 runs   (   13.84 ms per token,    72.24 tokens per second)
llama_perf_context_print:       total time =    2765.17 ms /   188 tokens
Llama.generate: 7 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_p

At iteration 0 Entropy:-0.0


Llama.generate: 7 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =     737.43 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =    3236.62 ms /   223 runs   (   14.51 ms per token,    68.90 tokens per second)
llama_perf_context_print:       total time =    3451.05 ms /   224 tokens
Llama.generate: 7 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =     737.43 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =    3557.23 ms /   256 runs   (   13.90 ms per token,    71.97 tokens per second)
llama_perf_context_print:       total time =    3799.40 ms /   257 tokens
Llama.generate: 7 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_p

At iteration 1 Entropy:-0.0


llama_perf_context_print:        load time =     737.43 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =    2982.39 ms /   214 runs   (   13.94 ms per token,    71.75 tokens per second)
llama_perf_context_print:       total time =    3152.50 ms /   215 tokens
Llama.generate: 7 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =     737.43 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =    3584.33 ms /   256 runs   (   14.00 ms per token,    71.42 tokens per second)
llama_perf_context_print:       total time =    3784.38 ms /   257 tokens
Llama.generate: 7 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =     737.43 ms
llama_perf_context_print: promp

...

At iteration 17 Entropy:-0.0


llama_perf_context_print:        load time =     737.43 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =    2354.40 ms /   169 runs   (   13.93 ms per token,    71.78 tokens per second)
llama_perf_context_print:       total time =    2501.80 ms /   170 tokens
Llama.generate: 7 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =     737.43 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =    2819.26 ms /   201 runs   (   14.03 ms per token,    71.30 tokens per second)
llama_perf_context_print:       total time =    3006.78 ms /   202 tokens
Llama.generate: 7 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =     737.43 ms
llama_perf_context_print: promp

At iteration 18 Entropy:-0.0


llama_perf_context_print:        load time =     737.43 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =    3366.06 ms /   240 runs   (   14.03 ms per token,    71.30 tokens per second)
llama_perf_context_print:       total time =    3557.72 ms /   241 tokens
Llama.generate: 7 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =     737.43 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =    3598.86 ms /   256 runs   (   14.06 ms per token,    71.13 tokens per second)
llama_perf_context_print:       total time =    3803.10 ms /   257 tokens
Llama.generate: 7 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =     737.43 ms
llama_perf_context_print: promp

At iteration 19 Entropy:-0.0


In [ ]:
prompt=final_prompt
semantic_response=[model.create_completion(prompt=prompt, temperature=temp, max_tokens=256)['choices'][0]['text'] for _ in range(5)]

entropy, labels=compute_semantic_entropy(semantic_response)
print(f"Entropy: {entropy: .3f}")

if entropy>1.5:
    print("Possible hallucination.")

else:
    print("Response likely grounded")


Llama.generate: 6 prefix-match hit, remaining 554 prompt tokens to eval
llama_perf_context_print:        load time =   14894.28 ms
llama_perf_context_print: prompt eval time =   14753.43 ms /   554 tokens (   26.63 ms per token,    37.55 tokens per second)
llama_perf_context_print:        eval time =   21498.60 ms /   255 runs   (   84.31 ms per token,    11.86 tokens per second)
llama_perf_context_print:       total time =   36652.24 ms /   809 tokens
Llama.generate: 559 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =   14894.28 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =     379.05 ms /     4 runs   (   94.76 ms per token,    10.55 tokens per second)
llama_perf_context_print:       total time =     385.26 ms /     5 tokens
Llama.generate: 559 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_con

Entropy:  0.971
Response likely grounded


In [ ]:
def ask_query():
    user_query=input("What is your question: ")
    response=model.create_completion(prompt=user_query, temperature=temp, max_tokens=256)['choices'][0]['text']
    print(response)

In [ ]:
ask_query()

NameError: name 'model' is not defined